Decompose text datasets with LLMs
+ Based on Zhipu Api.

Supported datasets:
+ trec-covid
+ msmarco (todo)

In [17]:
import csv
from zhipuai import ZhipuAI

my_api_key = "69a629a0f91455a08236a8dc5d74198b.0XT7BFKna9YEnJSP"

In [18]:
prompt_training = "你是一个自然语言的疑问句分解助手，请结合语义，将Q中的疑问句，分解成中符合语义的关键词组，以逗号为分隔。 \
特别注意：根据语义，**关键词组是一个短语，包含完整的语义，尽量不是单个的单词**；\
以下是示例：\
Q: \"how does the coronavirus respond to changes in the weather\", \
A: \"coronavirus respond to changes, changes in the weather\", \
Q: \"what causes death from Covid-19?\", \
A: \"causes of death, death from Covid-19\", \
Q: \"what are the guidelines for triaging patients infected with coronavirus?\", \
A: \"guidelines for triaging patients, infected with coronavirus\", \
Q: \"what kinds of complications related to COVID-19 are associated with hypertension?\", \
A: \"what complications, complications related to COVID-19, complications associated with hypertension\", \
Q: \"what are the health outcomes for children who contract COVID-19?\", \
A: \"health outcomes for children, children who contract COVID-19\", \
"

# Q: \"how has COVID-19 affected Canada\", \
# A: \"COVID-19, impact on Canada\", \

print(prompt_training)

你是一个自然语言的疑问句分解助手，请结合语义，将Q中的疑问句，分解成中符合语义的关键词组，以逗号为分隔。 特别注意：根据语义，**关键词组是一个短语，包含完整的语义，尽量不是单个的单词**；以下是示例：Q: "how does the coronavirus respond to changes in the weather", A: "coronavirus respond to changes, changes in the weather", Q: "what causes death from Covid-19?", A: "causes of death, death from Covid-19", Q: "what are the guidelines for triaging patients infected with coronavirus?", A: "guidelines for triaging patients, infected with coronavirus", Q: "what kinds of complications related to COVID-19 are associated with hypertension?", A: "what complications, complications related to COVID-19, complications associated with hypertension", Q: "what are the health outcomes for children who contract COVID-19?", A: "health outcomes for children, children who contract COVID-19", 


In [19]:
# prompt_inferring = '请根据以上的要求和示例，请给出以下Q的分解结果。输出格式：对于每一个Q，只输出一行分解后的子句 "A=分解后的句子"，不需要输出原句Q，子句之间以","分割，每两个A之间不要有空行。\
#     如果待分解句子中含有it和them等代词，则在分解的子句中，要结合句子含义将他们转换成对应的单词或词组；介词短语不能省略；\
#     每条分解后的子句都是有实际含义短语或短句，短句之间可以有单词相互重复，但尽量包含前后联系的语义信息，不是一个单独的词'

prompt_inferring = 'Q&A示例展示完毕，请给出以下Q的分解结果；输出格式：对于每一个Q，各回复一行分解后的子句"分解后的句子"，不要输出原句Q，每两行之间不要有空行。'

print(prompt_inferring)

Q&A示例展示完毕，请给出以下Q的分解结果；输出格式：对于每一个Q，各回复一行分解后的子句"分解后的句子"，不要输出原句Q，每两行之间不要有空行。


In [20]:
# for generation of trec-covid
 
client = ZhipuAI(api_key=my_api_key)

test_choice = 1
if test_choice:
    input_file = './input/queries-trec-covid.txt'
    output_file = './output/ans-trec-covid.txt'
else:
    input_file = './input/queries-msmarco.txt'
    output_file = './output/ans-msmarco.txt'

# prompt = input("user:")
begin = -50
batch_size = 50
while True:
    begin = begin + batch_size
    end = begin + batch_size
    infer = []

    print(begin)
    print(end)
    
    if(end > 50):
        break

    with open(input_file, 'r', encoding='utf-8') as file:
        for line_number, line in enumerate(file, start=1):
            if begin <= line_number <= end:
                infer.append(line.strip())
            elif line_number > end:
                break
    
    # print(infer)
    for i in infer:
        prompt_inferring = prompt_inferring + "Q: \"" + i + "\","
    
    prompt_inferring = prompt_inferring + "接下来请你完成分解任务，把疑问句Q分解成对应的**符合语义**的**包含原句信息**的**关键词组**。"

    # print(prompt_inferring)

    prompt = prompt_training + prompt_inferring
    # print(prompt)


    response = client.chat.completions.create(
        model="glm-4",  # model
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature = 0.3, # parameters
    )
    # answer = response.choices[0].message
    answer = response.choices[0].message.content
    print(answer)

    with open(output_file, 'a') as ans_file:
        ans_file.write(answer)
        ans_file.write("\n")

0
50
origin of COVID-19
coronavirus respond to changes in the weather
SARS-CoV2 infected people, develop immunity, cross protection
causes of death, death from Covid-19
drugs active against SARS-CoV or SARS-CoV-2, animal studies
types of rapid testing, testing for Covid-19
serological tests, detect antibodies, coronavirus
lack of testing availability, underreporting, true incidence of Covid-19
COVID-19 affected Canada
social distancing, impact on slowing spread of COVID-19
guidelines for triaging patients, infected with coronavirus
best practices, hospitals, home, maintaining quarantine
transmission routes of coronavirus
evidence related to COVID-19 super spreaders
coronavirus live outside the body
coronavirus remain stable on surfaces
clinical trials available, coronavirus
best masks, preventing infection by Covid-19
hand sanitizer, needed to destroy Covid-19
patients taking ACE inhibitors, increased risk for COVID-19
mortality rates overall, specific populations
cardiac complications